<a href="https://colab.research.google.com/github/vikasmech/TMDSI/blob/main/Data_prep_EDA_TMDSI_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
ls

drive/  sample_data/


In [3]:
cd drive/MyDrive/DSI/

/content/drive/MyDrive/DSI


In [4]:
ls

DATA/  reference/


In [5]:
cd DATA/

/content/drive/MyDrive/DSI/DATA


In [6]:
pwd

'/content/drive/MyDrive/DSI/DATA'

In [7]:
DATA_DIR = "/content/drive/MyDrive/DSI/DATA"

In [8]:
ls

natural-questions/  nq-validation.tsv
nq-counts.json      v1.0-simplified_nq-dev-all.jsonl.gz
nq-train.tsv        v1.0-simplified_simplified-nq-train.jsonl.gz


In [9]:
ls -rltah

total 9.8G
drwx------ 2 root root  4.0K Mar 28 19:40 natural-questions/
-rw------- 1 root root  4.3G Mar 28 20:05 nq-train.tsv
-rw------- 1 root root  101M Mar 28 20:15 nq-validation.tsv
-rw------- 1 root root    36 Mar 28 20:17 nq-counts.json
-rw------- 1 root root 1019M Mar 28 20:31 v1.0-simplified_nq-dev-all.jsonl.gz
-rw------- 1 root root  4.4G Mar 28 20:39 v1.0-simplified_simplified-nq-train.jsonl.gz


In [12]:
%tensorflow_version 2.x
import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
from bs4 import BeautifulSoup

In [13]:
ls -rltah

total 9.8G
drwx------ 2 root root  4.0K Mar 28 19:40 natural-questions/
-rw------- 1 root root  4.3G Mar 28 20:05 nq-train.tsv
-rw------- 1 root root  101M Mar 28 20:15 nq-validation.tsv
-rw------- 1 root root    36 Mar 28 20:17 nq-counts.json
-rw------- 1 root root 1019M Mar 28 20:31 v1.0-simplified_nq-dev-all.jsonl.gz
-rw------- 1 root root  4.4G Mar 28 20:39 v1.0-simplified_simplified-nq-train.jsonl.gz


In [18]:
import gzip
import json

# Data downloaded from https://ai.google.com/research/NaturalQuestions/download
# directory of Natural Questions data .
NQ_JSONL_DIR = DATA_DIR
NQ_SPLIT_FNAMES = {
    "train": "v1.0-simplified_simplified-nq-train.jsonl.gz",
    "validation": "v1.0-simplified_nq-dev-all.jsonl.gz"
}
nq_tsv_path = {
    "train": os.path.join(DATA_DIR, "nq-train.tsv"),
    "validation": os.path.join(DATA_DIR, "nq-validation.tsv")
}

In [37]:
for split, fname in NQ_SPLIT_FNAMES.items():
  print(split,fname)
  in_fname = os.path.join(NQ_JSONL_DIR, fname)
  with tf.io.gfile.GFile(in_fname, "rb") as infile:
    for line in gzip.open(infile):
      ex = json.loads(line)
      print(ex)
      print(ex.keys())
      if "document_text" in ex:
        html = ex["document_text"]
        soup = BeautifulSoup(html)
        print(soup.get_text())
      print("-"*80)

      break

train v1.0-simplified_simplified-nq-train.jsonl.gz
{'document_text': "Email marketing - Wikipedia <H1> Email marketing </H1> Jump to : navigation , search <Table> <Tr> <Td> </Td> <Td> ( hide ) This article has multiple issues . Please help improve it or discuss these issues on the talk page . ( Learn how and when to remove these template messages ) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed . ( September 2014 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research . Please improve it by verifying the claims made and adding inline citations . Statements consisting only of original research should be removed . ( January 2015 ) ( Learn how and when to remove this template message ) </Td> </Tr> </Table> ( Learn how and when t

# train v1.0-simplified_simplified-nq-train.jsonl.gz 
has following keys: 


-  'document_text', 
-  'long_answer_candidates', 
-  'question_text', 
-  'annotations', 
-  'document_url', 
-  'example_id'

where as 
# validation v1.0-simplified_nq-dev-all.jsonl.gz

- 'annotations',
- 'document_html', 
- 'document_title', 
- 'document_tokens',
- 'document_url', 
- 'example_id',
- 'long_answer_candidates', 
- 'question_text',
- 'question_tokens'

We can covert "document_tokens" to text based on [code](https://raw.githubusercontent.com/google-research-datasets/natural-questions/fb26a3073b1fe636c97302890a27b491d6530130/text_utils.py). [repo](https://github.com/google-research-datasets/natural-questions) 

```
  def _clean_token(token):
    """Returns token in which blanks are replaced with underscores.
    HTML table cell openers may contain blanks if they span multiple columns.
    There are also a very few unicode characters that are prepended with blanks.
    Args:
      token: Dictionary representation of token in original NQ format.
    Returns:
      String token.
    """
    return re.sub(u" ", "_", token["token"])

  text = " ".join([_clean_token(t) for t in nq_example["document_tokens"]])
```



In [34]:
!head nq-train.tsv

which is the most common use of opt-in e-mail marketing?	Email marketing-Wikipedia <H1> Email marketing </H1> Jump to:navigation, search <Table> <Tr> <Td> </Td> <Td> (hide) This article has multiple issues. Please help improve it or discuss these issues on the talk page. (Learn how and when to remove these template messages) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed. (September 2014) (Learn how and when to remove this template message) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research. Please improve it by verifying the claims made and adding inline citations. Statements consisting only of original research should be removed. (January 2015) (Learn how and when to remove this template message) </Td> </Tr> </Table> (Learn how and when to remove this template message) </Td> <